In [ ]:
!unzip drive/MyDrive/AIJ_2GIS_ru.zip
!unzip AIJ_2GIS_ru/AIJ_2gis.zip

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import torch.nn as nn
import torch

from torch.utils.data import Dataset
from torch.utils.data import DataLoader

from torchvision import transforms
from torchvision import models
from tqdm.notebook import tqdm

import random
from PIL import Image

import math

In [ ]:
device = torch.device("cuda")
random.seed(1)

In [ ]:
def get_classes():
    classes = []

    file_ = open("AIJ_2gis/train.csv", "r")
    for line in file_.readlines()[1:]:
        _, label = line.split(",")
        label = label.split("+")[0].rstrip("\n")

        if label not in classes:
            classes.append(label)
    
    return sorted(classes)

In [ ]:
class Data(Dataset):
    def __init__(self, i, j, seed, transform, device):
        super(Data, self).__init__()
        self.transform = transform
        self.device = device

        self.classes = get_classes()
        self.length = len(self.classes)

        file_ = open("AIJ_2gis/train.csv", "r")
        table = file_.readlines()[1:]

        random.seed(seed)
        random.shuffle(table)

        self.table = [line.split(",") for line in table[i:j]]
    
    def __len__(self):
        return len(self.table)
    
    def __getitem__(self, index):
        file_name, label = self.table[index]
        label = label.split("+")[0].rstrip("\n")
        
        img = Image.open("AIJ_2gis/" + file_name)
        img = self.transform(img).to(self.device)

        return img, self.get_tensor(label)

    def get_tensor(self, label):
        tensor = torch.zeros(self.length)
        tensor[self.classes.index(label)] = 1
        return tensor.to(self.device)

In [ ]:
train_data = Data(
    i=1, j=45000, seed=45, device=device,
    transform=transforms.Compose([
        transforms.Resize([55, 55]),
        transforms.RandomRotation((-5, 5)),
        
        transforms.Grayscale(num_output_channels=1),
        transforms.ToTensor(),
    ])
)

test_data = Data(
    i=45000, j=45700, seed=45, device=device,
    transform=transforms.Compose([
        transforms.Resize([55, 55]),

        transforms.Grayscale(num_output_channels=1),
        transforms.ToTensor(),
    ])
)

val_data = Data(
    i=45700, j=46060, seed=45, device=device,
    transform=transforms.Compose([
        transforms.Resize([55, 55]),

        transforms.Grayscale(num_output_channels=1),
        transforms.ToTensor(),
    ])
)

In [ ]:
train_dataloader = DataLoader(train_data, batch_size=50, shuffle=True)

In [ ]:
len(train_data.classes)

153

In [ ]:
def plot_history(train_history, val_history, title='loss'):
    plt.figure()
    plt.title('{}'.format(title))
    plt.plot(train_history, label='train', zorder=1)
    
    points = np.array(val_history)
    steps = list(range(0, len(train_history) + 1, int(len(train_history) / len(val_history))))[1:]
    
    plt.scatter(steps, val_history, marker='+', s=180, c='orange', label='val', zorder=2)
    plt.xlabel('train steps')
    
    plt.legend(loc='best')
    plt.grid()

    plt.show()

In [ ]:
class Net(nn.Module):
    def __init__(self, in_channels, n_classes):
        super(Net, self).__init__()
        self.model = models.densenet121(pretrained=False)
        self.model.features.conv0 = nn.Conv2d(in_channels, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
        self.model.classifier = nn.Linear(in_features=1024, out_features=n_classes, bias=True)
    
    def forward(self, x):
        x = self.model(x)
        return x

In [ ]:
def numel(m: torch.nn.Module, only_trainable: bool = False):
    """
    returns the total number of parameters used by `m` (only counting
    shared parameters once); if `only_trainable` is True, then only
    includes parameters with `requires_grad = True`
    """
    parameters = m.parameters()
    if only_trainable:
        parameters = list(p for p in parameters if p.requires_grad)
    unique = dict((p.data_ptr(), p) for p in parameters).values()
    return sum(p.numel() for p in unique)

In [ ]:
model = Net(in_channels=1, n_classes=train_data.length)
model.to(device)

In [ ]:
print(numel(model, False))

7104409


In [ ]:
def checkpoint(accuracy):
    PATH = f"./drive/MyDrive/AIJ_2gis/checkpoints/checkpoint_accuracy-{accuracy}.pth"
    torch.save(model.state_dict(), PATH)
    print("checkpoint created - " + PATH)

In [ ]:
def train(model, criterion, optimizer, train_dataloader, scheduler1, scheduler2, NUM_EPOCH=20):
    running_loss = .0
    train_size = .0
    
    loss_log = []
    accuracy_log = []

    correct = 0
    count = 0

    model.train()

    for epoch in range(NUM_EPOCH):
        tq = tqdm(enumerate(train_dataloader), total=math.ceil(len(train_data) / train_dataloader.batch_size), ascii=True)
        tq.set_description("Epoch #" + str(epoch + 1))

        for i, (x, y) in tq:
            optimizer.zero_grad()
            out = nn.LogSoftmax(model(x)).dim

            loss = criterion(out, y.argmax(dim=1))
            loss.backward()

            running_loss = loss.item()
            train_size += out.size(0)

            optimizer.step()

            for i in range(int(out.shape[0])):
                if list(out[i]).index(max(out[i])) == list(y[i]).index(max(y[i])):
                    correct += 1
            
            count += int(out.shape[0])

            loss_log.append(loss.data / out.size(0))
            accuracy_log.append(correct / count)


            tq.set_postfix({
                "loss": (running_loss / train_size) * 1000,
                "accuracy": correct / count,
            })
        
        scheduler1.step()
        scheduler2.step()
        
        val_accuracy = evaluate(model, val_data)
        print("Val accuracy: %.3f" % (val_accuracy,) )
        checkpoint(val_accuracy)

    
    return loss_log, accuracy_log

In [ ]:
def evaluate(model, data):
    model.eval()
    
    correct = 0
    for x, y in data:
        out = nn.LogSoftmax(model(x.view(1, 1, 55, 55))).dim[0]
        if list(out).index(max(out)) == list(y).index(max(y)):
            correct += 1
    return correct / len(data)

In [110]:
model.load_state_dict(torch.load("./drive/MyDrive/AIJ_2gis/checkpoints/checkpoint_accuracy-0.9944444444444445.pth"))

<All keys matched successfully>

In [112]:
evaluate(model, val_data) 

0.9944444444444445

In [ ]:
optimizer = torch.optim.Adamax(model.parameters(), lr=0.01)
scheduler1 = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.9)
scheduler2 = torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones=[30,80], gamma=0.1)

criterion = nn.CrossEntropyLoss()

In [109]:
log = train(model, criterion, optimizer, train_dataloader, scheduler1, scheduler2, 1)


Val accuracy: 0.989
checkpoint created - ./drive/MyDrive/AIJ_2gis/checkpoints/checkpoint_accuracy-0.9888888888888889.pth


In [113]:
torch.onnx.export(model, torch.randn(1, 1, 55, 55, device='cuda'), "./drive/MyDrive/AIJ_2gis/model.onnx", verbose=True, input_names=["actual_input_1"]+ [ "learned_%d" % i for i in range(16) ], opset_version=10, output_names=["otput1"])

graph(%actual_input_1 : Float(1, 1, 55, 55, strides=[3025, 3025, 55, 1], requires_grad=0, device=cuda:0),
      %learned_6 : Float(64, strides=[1], requires_grad=1, device=cuda:0),
      %learned_7 : Float(64, strides=[1], requires_grad=1, device=cuda:0),
      %learned_8 : Float(64, strides=[1], requires_grad=0, device=cuda:0),
      %learned_9 : Float(64, strides=[1], requires_grad=0, device=cuda:0),
      %model.features.denseblock1.denselayer1.conv2.weight : Float(32, 128, 3, 3, strides=[1152, 9, 3, 1], requires_grad=1, device=cuda:0),
      %model.features.denseblock1.denselayer2.norm1.weight : Float(96, strides=[1], requires_grad=1, device=cuda:0),
      %model.features.denseblock1.denselayer2.norm1.bias : Float(96, strides=[1], requires_grad=1, device=cuda:0),
      %model.features.denseblock1.denselayer2.norm1.running_mean : Float(96, strides=[1], requires_grad=0, device=cuda:0),
      %model.features.denseblock1.denselayer2.norm1.running_var : Float(96, strides=[1], requires_g

In [ ]:
out = nn.LogSoftmax(model(train_data[0][0].view(1, 1, 55, 55))).dim[0]
list(out).index(max(out)), list(train_data[0][1]).index(max(train_data[0][1]))